## dbt modeling
vid mindre projekt brukar man använda staging och refined, och ibland intermediate models
vid större mer komplexa projekt brukar man använda src, dim, fct och mart models

- src (source model) - samma som staging   
    - Innehåller rådata-källor, som filer, databaser, APIs osv.   
    - Första steget i ETL.    
    - Används för att skapa dim och src modeller. 
    - src-mappen innehåller en scources.yml fil som innehåller beskrivningar/instruktioner för alla sources.      

- dim (dimensional model) - samma som intermediate
    - Används för typ products, customers osv, saker som behövs för att skapa ett fact-table.     
    - Används för att filtrera, gruppera, sortera eller ge kontext till faktadata.    
    - Typiskt har dessa en primärnyckel (CustomerID, ProductID).      

- fct (fact model) - samma som intermediate
    - Används för analys, siffror osv, typ sales, revenue.    
    - Connectar ihop flera dim-models med mätbar data.    
    - Skapas via ett ***star schema*** med hjälp av FK och PK från dim-tables för att få fram analyserbara metrics.     
    - Central tabell i ett star schema.   
    - Innehåller ofta: summor, antal, medelvärden, etc.   

- mart (specifikt affärsanpassad model) - samma som refined
    - Färdigtransformerade affärsanpassade modeller för specifika ändamål.    
    - Fokus på en specifik domän eller användargrupp (t.ex. försäljning, marknadsföring, ekonomi).    
    - Används direkt för rapportering, dashboards eller ad-hoc-analyser.      
    - Kan innehålla kombinationer av fakta och dimensioner från olika källor.     

## tables vs views(data materialization)
***Consideration of data warehouse:***
- Minimize storage & computing costs
- Minimize downtime
    - Consider the tradeoffs

***Decision making:***
- Team 1: regularly updates dashboard every day.
- Team 2: sometimes wants a dashboard to solve a specifik problem.

***Table for team 1:***
- Needs to regularly update dashboard, will need the data for future uses, dont want downtime when refreshing dashboard.
- Wort putting money for storage cost, but not computing cost

***View for team 2:***
- Dont need regular updates for dashboard, only needs the data for a short period, dont care alot about downtime.
- Wort putting money for computing cost, but not storage cost


***Views:*** 
- Använder ett script/query från ett table och får fram ett result set, tar ingen plats i databasen.
- Minimerar storage cost, men ökar computing cost.
- Visar informationen.
- Bra när datan inte ska användas kontinueligt.
- När datan inte behöver sparas och användas ofta
- Virtuella tabeller – sparar inte data fysiskt, utan körs varje gång man frågar efter dem.
- Fördelar:
    - Minimal storage cost – ingen data dupliceras.
    - Alltid färsk data – visar senaste datan i källtabellerna.
- Nackdelar:
    - Högre computing cost – måste beräknas varje gång den används.
    - Kan vara långsamma om de används i komplexa queries ofta.
- Bra för:
    - Ad-hoc-analyser.
    - Data som inte används frekvent.
    - Transformationer du inte vill materialisera än.


***Tables:***
- Skapar ett table, kräver inga querys för result sets, men tar mycket mer plats i databasen.
- Ökar storage cost, men minimerar computing cost.
- Sparar information.
- Bra när datan används ofta/dagligen.
- Datan är värd att sparas då den används mycket. 
- Fysiskt lagrade tabeller – datan skrivs till lagring och finns tillgänglig direkt.
- Fördelar:
    - Låg computing cost – snabb åtkomst, inget behov att köra komplexa queries varje gång.
    - Bra för prestanda i dashboards och rapporter.
- Nackdelar:
    - Högre storage cost – data dupliceras.
    - Behöver uppdateras regelbundet för att hålla sig aktuell.
- Bra för:
    - Frekvent använda datasets.
    - Dashboards som laddas dagligen.
    - Data som förändras långsamt eller enligt schema (t.ex. en gång per natt).

***ephemeral***
skapar ingen faktisk table eller view som sparas/lagras och tar plats i databasen, bra att använda till source för att kunna läsa in data från till andra models

## surrogate key
kolla efter **finest grain** , alltså kolumnen med flest olika värden bland dom kolumner du valt som hör ihop på något sätt, gör denna till surrogate key med hjälp av macro-funktionen:
```python
{{dbt_utils.dbt_utils.generate_surrogate_key(['column_name'])}}
```
det går också att para ihop flera columner för att skapa en surrogate key, denna används sedan som en primary key

## SQL-funktioner
***coalesce***
används som replace för nullvärden, med kolumnnamn följt av vad du vill ersätta nullvärdet med:
```sql
coalesce(duration, 'ej angiven') as duration
```

***dduplicate***
för att ta bort dubletter vid group by använd max() som aggregering

## packages
för att lägga till packages som kan innehålla tex macro-funktioner för att generera en surrogate key, så skapa en packages.yml fil i dbt-foldern och lägg till package, sedan kör kommandot dbt deps i terminalen för att läsa in paketen och sedan dbt run:
```bash
"i terminalen:"
$ touch packages.yml

"i filen:"
packages:
  - package: dbt-labs/dbt_utils
    version: 1.3.0

"i terminalen:"
$ dbt deps
$ dbt run

"kan hända att du även behöver starta om vscode efter detta"
```
